# Sequential Rigid Body Object Stacking

A greedy, physics-based next best object pose planning algorithm 

In [5]:
import pybullet as p
import os 
import time
import pybullet_data
import threading
import numpy as np
import scipy
import trimesh

from Rocks.TowerCenterOfMass import convex_decomposition_center_of_mass_calculation


In [6]:
PATH_TO_URDF_DEFINITIONS = './Rocks/SingleRockFiles_Processed'

def get_rock_definition_directories(): 
    return [fname for fname in os.listdir(PATH_TO_URDF_DEFINITIONS) if 'rock' in fname]

rock_dirs = get_rock_definition_directories()
print(f"we have data for {len(rock_dirs)} objects")

we have data for 31 objects


In [3]:
#Setup the physics engine. 
physicsClient = p.connect(p.GUI) #or p.DIRECT for non-graphical version
p.setAdditionalSearchPath(pybullet_data.getDataPath()) #optionally
p.setGravity(0,0,0) #world has no universal gravity, we apply gravity to individual objects as need be 

#Create a platform upon which we will perform stacking 
planeId = p.loadURDF("plane.urdf", basePosition=[0,0,0])

In [10]:
#the number of rocks total 
rock_dirs = get_rock_definition_directories()
nrocks = len(rock_dirs)

#the number of rocks we place in the scene 
use_nrocks = 30

#select some randomized subset of rocks
#these are the rocks we will place in the physics simulator 
rockindex = [i for i in range(nrocks)]
np.random.seed(42)
np.random.shuffle(rockindex)

rock_dirs = [rock_dirs[i] for i in rockindex[:use_nrocks]]

#Paths to the urdf files which define our rocks 
rock_paths = [f"{PATH_TO_URDF_DEFINITIONS}/{rdir}/{rdir}.urdf" for rdir in rock_dirs]

print("we are using the following rocks definitions for our simulation\n")
print('\n'.join(rock_paths))

we are using the following rocks definitions for our simulation

./Rocks/SingleRockFiles_Processed/rock25/rock25.urdf
./Rocks/SingleRockFiles_Processed/rock28/rock28.urdf
./Rocks/SingleRockFiles_Processed/rock12/rock12.urdf
./Rocks/SingleRockFiles_Processed/rock1/rock1.urdf
./Rocks/SingleRockFiles_Processed/rock18/rock18.urdf
./Rocks/SingleRockFiles_Processed/rock20/rock20.urdf
./Rocks/SingleRockFiles_Processed/rock14/rock14.urdf
./Rocks/SingleRockFiles_Processed/rock15/rock15.urdf
./Rocks/SingleRockFiles_Processed/rock19/rock19.urdf
./Rocks/SingleRockFiles_Processed/rock5/rock5.urdf
./Rocks/SingleRockFiles_Processed/rock16/rock16.urdf
./Rocks/SingleRockFiles_Processed/rock30/rock30.urdf
./Rocks/SingleRockFiles_Processed/rock29/rock29.urdf
./Rocks/SingleRockFiles_Processed/rock10/rock10.urdf
./Rocks/SingleRockFiles_Processed/rock26/rock26.urdf
./Rocks/SingleRockFiles_Processed/rock0/rock0.urdf
./Rocks/SingleRockFiles_Processed/rock2/rock2.urdf
./Rocks/SingleRockFiles_Processed/rock3/ro

In [31]:
# def get_object_bounding_box(obj_id): 
#     #we are interested in the maximal bounding box of an object 
#     aabbMin, aabbMax = p.getAABB(obj_id)
#     return aabbMin, aabbMax 

# def get_xyz_com_from_aabb(aabbMin, aabbMax): 
#     '''
#     Currently, we are approximating a 3D axis aligned 
#     bounding box and computing the centroid. 
#     '''
#     return [
#         (aabbMin[0]+aabbMax[0])/2,
#         (aabbMin[1]+aabbMax[1])/2,
#         (aabbMin[2]+aabbMax[2])/2,
#     ]

class RockTower: 
    '''
    Abstraction to encapsulate information about the tower of rocks. 
    
    [Rock] - rocks: 
        A list of rock objects 
    '''
    
    def __init__(self): 
        self.rocks = []
        
        
class Rock: 
    '''
    Abstraction representing our physics simulator rock object
    
    String - simid: 
        The id used to reference the simulation object that represents this rock
    [scipy.spatial.ConvexHull] - pieces: 
        Collection of convex hulls that when joined together forms the rock. 
    Float - com: 
        the center of mass of the rock, which we approximate as the geometric 
        centroid over the set of convex hulls. 
    [Float,Float,Float,Float] - ori_quaternion: 
        The original (x,y,z) position and angular orientation w of the object 
        We compute transforms relative to this local origin point 
    
    '''
    
    def __init__(self, rockid, hulls): 
        self.rockid = rockid
        self.hulls = hulls 
        
        #derived from hulls
        self.com = self.get_com() # rockid and pieces must be set before this call 
        
        # Original position/orientation in [x,y,z,w] form 
        self.ori_quaternion = p.getBasePositionAndOrientation(self.rockid) 
        
    def get_rockid(self): 
        return self.rockid

    def get_com(self): 
        #get the center of mass over all hulls that comprise the rock 
        return convex_decomposition_center_of_mass_calculation(self.hulls)
    
    def get_mass(self): 
        #volume is proportional to mass so this is acceptable
        return np.sum([hull.volume for hull in self.hulls])
    
    def get_ori_quaternion(self): 
        return self.ori_quaternion
    
    def get_ori_position(self): 
        return self.ori_quaternion[:3]
        
    def get_ori_orientation(self): 
        return self.ori_quaternion[3]
    
    def get_quaternion(self): 
        return p.getBasePositionAndOrientation(self.rockid)
    
    def get_position(self): 
        return self.get_quaternion(self.rockid)[:3]
        
    def get_orientation(self): 
        return self.get_quaternion(self.rockid)[3]
        
        

In [4]:
def spawn_rock(rock_path): 
    '''
    Load a rock from a .urdf definition file into pyBullet physics simulation. 
    Returns: 
        String: The id of the created object 
    '''
    rockid = p.loadURDF(rock_path, globalScaling=.04)
    return rockid 

def spawn_rocks(rock_paths): 
    '''
    Load a collection of rocks into the simulation
    Returns: 
        [String]: The ids of the created objects 
    ''' 
    return [create_rock_in_simulation(rpath) for rpath in rock_paths]
        
rock_ids = spawn_rocks(rock_paths)

NameError: name 'rock_paths' is not defined

In [9]:
# while 1: 
#     p.stepSimulation()
#     for idi in rock_ids: 
#         print(idi)
#         print(compute_center_of_mass(idi))
#         print()
#     break
#     time.sleep(1./240.)